In [35]:
import pandas as pd
import json
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [9]:
df = pd.read_json('cell_phones_accessories.json', lines=True)

In [10]:
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,[{'small_image_url': 'https://images-na.ssl-im...,B08L6L3X1S,B08L6L3X1S,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-01-30 22:07:31.196,0,True
1,5,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,[],B079BPGF6C,B079BPGF6C,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-08-16 18:18:37.349,2,True
2,5,Worked but took an hour to install,Overall very happy with the end result. If you...,[{'small_image_url': 'https://m.media-amazon.c...,B088DR7Z5B,B0BBGGC8F2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2021-08-17 21:21:44.798,3,True
3,4,Decent,Lasted about 9 months then the lock button bro...,[{'small_image_url': 'https://images-na.ssl-im...,B07XRDHDNQ,B07XRDHDNQ,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2020-05-26 05:14:42.910,0,True
4,5,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,[],B00A8ZDL9Y,B00A8ZDL9Y,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2014-08-25 19:23:08.000,0,True
...,...,...,...,...,...,...,...,...,...,...
999995,3,First of all just an FYI for those wondering y...,First of all just an FYI for those wondering y...,[],B00GGY8PAQ,B00GGY8PAQ,AH6G2UJ3CJ4PIO56IRAOF62GRWZQ,2014-12-27 22:07:45.000,1,True
999996,5,Very good protection.,Looks good works well. Nothing else I expect.,[],B07H11F2GG,B07H11F2GG,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,2019-08-18 03:14:36.662,0,True
999997,5,Good charging cord,Nice and long and braided coating looks like i...,[],B01CY1CUB4,B01CY1CUB4,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,2016-06-16 04:42:14.000,0,True
999998,5,Perfect for iPhone! Love this!,This is perfect for my iPhone. I've bought oth...,[],B00GJ36AZG,B00GJ36AZG,AE2P2I3OV4K2N23GSAZADUCJ3RLQ,2015-06-27 00:32:20.000,1,True


In [11]:
df['verified_purchase'].value_counts()

verified_purchase
True     937676
False     62324
Name: count, dtype: int64

In [20]:
df = df.loc[df['verified_purchase'] != False]
df = df.reset_index(drop=True)
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,[{'small_image_url': 'https://images-na.ssl-im...,B08L6L3X1S,B08L6L3X1S,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-01-30 22:07:31.196,0,True
1,5,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,[],B079BPGF6C,B079BPGF6C,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-08-16 18:18:37.349,2,True
2,5,Worked but took an hour to install,Overall very happy with the end result. If you...,[{'small_image_url': 'https://m.media-amazon.c...,B088DR7Z5B,B0BBGGC8F2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2021-08-17 21:21:44.798,3,True
3,4,Decent,Lasted about 9 months then the lock button bro...,[{'small_image_url': 'https://images-na.ssl-im...,B07XRDHDNQ,B07XRDHDNQ,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2020-05-26 05:14:42.910,0,True
4,5,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,[],B00A8ZDL9Y,B00A8ZDL9Y,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2014-08-25 19:23:08.000,0,True
...,...,...,...,...,...,...,...,...,...,...
937671,5,Five Stars,Works well.,[],B00D3TAV5Y,B00D3TAV5Y,AHQSE5R5DERCA6VNSYAOTN3YAWDQ,2015-03-09 12:42:42.000,0,True
937672,3,First of all just an FYI for those wondering y...,First of all just an FYI for those wondering y...,[],B00GGY8PAQ,B00GGY8PAQ,AH6G2UJ3CJ4PIO56IRAOF62GRWZQ,2014-12-27 22:07:45.000,1,True
937673,5,Very good protection.,Looks good works well. Nothing else I expect.,[],B07H11F2GG,B07H11F2GG,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,2019-08-18 03:14:36.662,0,True
937674,5,Good charging cord,Nice and long and braided coating looks like i...,[],B01CY1CUB4,B01CY1CUB4,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,2016-06-16 04:42:14.000,0,True


In [21]:
df['verified_purchase'].value_counts()

verified_purchase
True    937676
Name: count, dtype: int64

In [22]:
'''
CountVectorizer
'''

'\nCountVectorizer\n'

In [25]:
review_vec = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize) 
review_counts = review_vec.fit_transform(df['text'])
review_counts

C:\Users\abhin\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<937676x157089 sparse matrix of type '<class 'numpy.int64'>'
	with 27473912 stored elements in Compressed Sparse Row format>

In [26]:
'''
80-20 train test split approach
'''

'\n80-20 train test split approach\n'

In [32]:
x_train, x_test, y_train, y_test = train_test_split(review_counts, df['rating'], test_size=0.2)

In [33]:
nb_model = MultinomialNB()

nb_model.fit(x_train, y_train)
y_pred = nb_model.predict(x_test)

cf_matrix = confusion_matrix(y_test, y_pred)

print("---------------------------------------------------------------------------------------------------------------------")
print("80-20 split\n")
print("Accuracy:", accuracy_score(y_test, y_pred), "\n")
print(cf_matrix)
print("---------------------------------------------------------------------------------------------------------------------")

---------------------------------------------------------------------------------------------------------------------
80-20 split

Accuracy: 0.702211841993004 

[[ 12535    608   1393    791   3815]
 [  3862    616   1982   1259   2855]
 [  2539    464   3035   3480   5419]
 [  1125    170   1518   6772  14237]
 [  2107    207    985   7030 108732]]
---------------------------------------------------------------------------------------------------------------------


In [ ]:
# svm_model = SVC()

# svm_model.fit(x_train, y_train)
# y_pred = svm_model.predict(x_test)

# cf_matrix = confusion_matrix(y_test, y_pred)

# print("---------------------------------------------------------------------------------------------------------------------")
# print("80-20 split\n")
# print("Accuracy:", accuracy_score(y_test, y_pred), "\n")
# print(cf_matrix)
# print("---------------------------------------------------------------------------------------------------------------------")

In [ ]:
# lr_model = LogisticRegression(max_iter=1000)

# lr_model.fit(x_train, y_train)
# y_pred = lr_model.predict(x_test)

# cf_matrix = confusion_matrix(y_test, y_pred)

# print("---------------------------------------------------------------------------------------------------------------------")
# print("80-20 split\n")
# print("Accuracy:", accuracy_score(y_test, y_pred), "\n")
# print(cf_matrix)
# print("---------------------------------------------------------------------------------------------------------------------")